## Tracks data analysis

Let's assert tracks data analysis by taking a first inspection of the track dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from os import path
dataset_path = path.join('..', 'dataset', 'tracks.csv')
df = pd.read_csv(dataset_path, sep=',')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11166 entries, 0 to 11165
Data columns (total 45 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    11166 non-null  object 
 1   id_artist             11166 non-null  object 
 2   name_artist           11166 non-null  object 
 3   full_title            11166 non-null  object 
 4   title                 11166 non-null  object 
 5   featured_artists      3517 non-null   object 
 6   primary_artist        11166 non-null  object 
 7   language              11061 non-null  object 
 8   album                 9652 non-null   object 
 9   stats_pageviews       4642 non-null   float64
 10  swear_IT              11166 non-null  int64  
 11  swear_EN              11166 non-null  int64  
 12  swear_IT_words        11166 non-null  object 
 13  swear_EN_words        11166 non-null  object 
 14  year                  10766 non-null  object 
 15  month              

year, month, day, n_sentence, n_tokens, disc_numer, track_number, popularity should all be int64 (or at least float64: year and popularity are object), explicit should be bool

validation function checking expected types validity

In [3]:
def check_type_validity(value, expected_type):
    return not isinstance(value, expected_type)

### id

In [4]:
invalid_elems = df[df['id'].apply(check_type_validity, expected_type=str)]
print(invalid_elems['id'])

nan_indexes = df.index[df['id'].isna()].tolist()
print(f"number of missing values in id: {len(nan_indexes)}")

unique_ids = df['id'].nunique()
print(f"number of unique ids: {unique_ids}")

Series([], Name: id, dtype: object)
number of missing values in id: 0
number of unique ids: 11093


there are no missing id values in the dataset.
all the ids are strings with the following format "TR{unique_num_id}".
there are less unique ids than the number of rows (11093 < 11166), let's check if they come from duplicate songs or there are different songs with the same id

In [5]:
not_unique_ids = df['id'].value_counts()
not_unique_ids = not_unique_ids[not_unique_ids > 1]
print(f"there are {len(not_unique_ids)} not unique ids:")
print(not_unique_ids)

not_unique_ids_list = not_unique_ids.index.tolist()

identical_duplicates = []
different_duplicates = []
mixed_duplicates = [] #ids with both identical and different rows

for dup_id in not_unique_ids_list:
    duplicate_rows = df[df['id'] == dup_id]
    num_occurrences = len(duplicate_rows)
    
    unique_songs = duplicate_rows[['name_artist', 'title']].drop_duplicates()
    num_unique = len(unique_songs)

    if num_unique == 1:
        identical_duplicates.append(dup_id) #duplicate rows
    elif num_unique == num_occurrences:
        different_duplicates.append(dup_id) #different songs
    else:
        mixed_duplicates.append(dup_id) #duplicate rows and different songs

print(f"{len(identical_duplicates)} come from duplicate songs, ",
      f"{len(different_duplicates)} come from different songs with the same id, ",
      f"{len(mixed_duplicates)} have both duplicate and different songs")

there are 71 not unique ids:
id
TR367132    4
TR978886    2
TR987615    2
TR690925    2
TR772702    2
           ..
TR245683    2
TR903275    2
TR679972    2
TR247772    2
TR261964    2
Name: count, Length: 71, dtype: int64
0 come from duplicate songs,  70 come from different songs with the same id,  1 have both duplicate and different songs


of the 71 not unique ids, 70 refers to different songs, meaning that there are no duplicate songs respect to those ids, but there are ids which refers to multiple songs.

by analyzing more in depth the case with both duplicate and different songs:

In [6]:
for dup_id in mixed_duplicates: #iterating for a "general" solution, but in this case wouldn't be necessary
    duplicate_artist_song = df[df['id'] == dup_id]
    unique_songs = duplicate_artist_song[['name_artist', 'title']].drop_duplicates()
    print(f"for id {dup_id}, there are {len(unique_songs)} unique songs:")
    
    for idx, row in unique_songs.iterrows():
        song_rows = duplicate_artist_song[
            (duplicate_artist_song['name_artist'] == row['name_artist']) &
            (duplicate_artist_song['title'] == row['title'])]
        
        print(f"\t- '{row['title']}' by artist: {row['name_artist']}, occuring {len(song_rows)} times")
        
        if len(song_rows) != 1: #more than one occurrence
            first_row = song_rows.iloc[0]
            all_identical = True
            differing_columns = []

            for col in song_rows.columns:
                unique_values = song_rows[col].nunique()
                if unique_values > 1:
                    all_identical = False
                    differing_columns.append(col)
            
            if not all_identical:
                print(f"\t\tdiffering columns: {differing_columns}\n")
                
                #for col in differing_columns: #uncomment this for see values of different rows
                        #values = song_rows[col].tolist()
                        #print(f"\t\t• {col}:")
                        #for i, val in enumerate(values):
                            #print(f"\t\t\tRow {i+1}: {val}")

for id TR367132, there are 2 unique songs:
	- 'BUGIE' by artist: Madame, occuring 2 times
		differing columns: ['year', 'album_name', 'album_release_date', 'album_type', 'track_number', 'duration_ms', 'popularity', 'album_image', 'id_album']

	- '​sentimi' by artist: Madame, occuring 2 times
		differing columns: ['album_name', 'album_release_date', 'album_type', 'track_number', 'duration_ms', 'popularity', 'album_image', 'id_album']



so that id not only refers to 2 different songs, but each occurrence of the song (2) has different values. based on the values of different rows, it's possible to cut the additional wrong records

### id_artist

In [7]:
invalid_elems = df[df['id_artist'].apply(check_type_validity, expected_type=str)]
print(invalid_elems['id_artist'])

nan_indexes = df.index[df['id_artist'].isna()].tolist()
print(f"number of missing values in id_artist: {len(nan_indexes)}")

id_to_names = df.groupby('id_artist')['name_artist'].nunique()
multiple_names = id_to_names[id_to_names > 1]

if len(multiple_names) == 0:
    print("each id_artist corresponds to exactly one name_artist")

Series([], Name: id_artist, dtype: object)
number of missing values in id_artist: 0
each id_artist corresponds to exactly one name_artist


there are no missing id_artist values in the dataset. all the id_artists are strings with the following format "ART{unique_num_id}". each id_artist corresponds to exactly one name_artist

### id_artist

In [8]:
invalid_elems = df[df['name_artist'].apply(check_type_validity, expected_type=str)]
print(invalid_elems['name_artist'])

nan_indexes = df.index[df['name_artist'].isna()].tolist()
print(f"number of missing values in name_artist: {len(nan_indexes)}")

name_to_ids  = df.groupby('name_artist')['id_artist'].nunique()
multiple_ids  = name_to_ids [name_to_ids  > 1]

if len(multiple_names) == 0:
    print("each name_artist corresponds to exactly one id_artist")

Series([], Name: name_artist, dtype: object)
number of missing values in name_artist: 0
each name_artist corresponds to exactly one id_artist


there are no missing name_artist values in the dataset. all the name_artists are strings with the following format "ART{unique_num_id}". each name_artist corresponds to exactly one id_artist

### full_title

In [9]:
invalid_elems = df[df['full_title'].apply(check_type_validity, expected_type=str)]
print(invalid_elems['full_title'])

nan_indexes = df.index[df['full_title'].isna()].tolist()
print(f"number of missing values in full_title: {len(nan_indexes)}")

Series([], Name: full_title, dtype: object)
number of missing values in full_title: 0


there are no missing full_title values in the dataset. 
all the full_titles are strings with the following format: "{title} by {primary_artist} (Ft. {name_artist_1}, {...}, {name_artist_n-1} & {name_artist_n})". let's check if name_artist is inside every full title

In [10]:
#normalizing non-breaking space and quotes
df['full_title'] = (df['full_title']
                    .str.replace('\xa0', ' ', regex=False)    #non-breaking space
                    .str.replace('\u2019', "'", regex=False)  #right single quote
                    .str.replace('\u2018', "'", regex=False)  #left single quote
                    .str.replace('\u201c', '"', regex=False)  #left double quote
                    .str.replace('\u201d', '"', regex=False)) #right double quote

#(.+?) -> capture everything; (?:\s*\(|$) -> stop at either " (" or end of string
extracted_artist = df['full_title'].str.extract(r' by (.+?)(?:\s*\(|$)', expand=False).str.strip()

mismatches = df[extracted_artist != df['primary_artist']]
print(f"number of primary_artist mismatches inside full_title: {len(mismatches)}")

if len(mismatches) > 0:
    print("sample mismatches:")
    mismatches_display = mismatches.copy()
    mismatches_display['extracted_artist'] = extracted_artist[mismatches.index]
    display(mismatches_display[['full_title', 'primary_artist', 'extracted_artist']].head(20))

number of primary_artist mismatches inside full_title: 844
sample mismatches:


,full_title,primary_artist,extracted_artist
36,"Mattonelle* by Rosa Chemical, Mehdi (ITA), Fri...",Rosa Chemical,"Rosa Chemical, Mehdi"
129,a me mi piace by Alfa & Manu Chao,Alfa,Alfa & Manu Chao
223,EGLI È IL RE by thasup & Mara Sattei,thasup,thasup & Mara Sattei
225,POSTO MIO by thasup & Mara Sattei,thasup,thasup & Mara Sattei
227,​r()t()nda by thasup & Tiziano Ferro,thasup,thasup & Tiziano Ferro
231,SO CHE CI SEI by thasup & Mara Sattei,thasup,thasup & Mara Sattei
235,BLESS SU BLESS by thasup & Mara Sattei,thasup,thasup & Mara Sattei
238,ONE KING by thasup & Mara Sattei,thasup,thasup & Mara Sattei
241,COME POLVERE by thasup & Mara Sattei,thasup,thasup & Mara Sattei
244,OCCHI MIEI by thasup & Mara Sattei,thasup,thasup & Mara Sattei


In [11]:
multi_artist_indicators = ['&', ',', ' and ']

has_separator = df['primary_artist'].str.contains('|'.join(multi_artist_indicators), case=False, na=False)
potential_multi = df[has_separator]

print(f"records with single artist: {len(df) - len(potential_multi)}")

records with single artist: 11166


The name_artist column contains only one artist per track, even when multiple artists collaborated (excluding featured artists), likely due to a data collection system limitation. while some tracks include non-rapper collaborators not in the dataset (e.g. Tiziano Ferro, Mara Sattei), which is expected, other tracks list only one rapper despite having multiple rapper collaborators who are present in the dataset (e.g. Ensi in Cemento). this causes artist-based analyses to undercount songs, as tracks where an artist collaborated but wasn't listed as the primary artist will be excluded from their catalog.

### title

In [12]:
invalid_elems = df[df['title'].apply(check_type_validity, expected_type=str)]
print(invalid_elems['title'])

nan_indexes = df.index[df['title'].isna()].tolist()
print(f"number of missing values in title: {len(nan_indexes)}")

Series([], Name: title, dtype: object)
number of missing values in title: 0


In [13]:
df['title'] = (df['title']
                .str.replace('\xa0', ' ', regex=False)
                .str.replace('\u2019', "'", regex=False)
                .str.replace('\u2018', "'", regex=False)
                .str.replace('\u201c', '"', regex=False)
                .str.replace('\u201d', '"', regex=False))

# ^(.+?) -> capture from start; \s+by\s+ -> stop at " by " (with flexible whitespace)
extracted_title = df['full_title'].str.extract(r'^(.+?)\s+by\s+', expand=False).str.strip()

mismatches = df[df['title'] != extracted_title]
print(f"number of title mismatches inside full_title: {len(mismatches)}")

if len(mismatches) > 0:
    print("remaining mismatches (likely titles containing 'by'):\n")
    mismatches_display = mismatches.copy()
    mismatches_display['extracted_title'] = extracted_title[mismatches.index]
    display(mismatches_display[['full_title', 'title', 'extracted_title']].head(10))

number of title mismatches inside full_title: 4
remaining mismatches (likely titles containing 'by'):



,full_title,title,extracted_title
2513,Bolo by Night RMX by Inoki,Bolo by Night RMX,Bolo
2578,Bolo by Night RMX (NewKingzTape) by Inoki (Ft....,Bolo by Night RMX (NewKingzTape),Bolo
2600,Bolo by Night (NewKingz RMX) by Inoki (Ft. Lil...,Bolo by Night (NewKingz RMX),Bolo
2642,Barona by Night (Red Bull Culture Clash dubpla...,Barona by Night (Red Bull Culture Clash dubplate),Barona


title is contained inside full_title with no errors. the missmatches appearing are titles containing the word 'by', which is what's used from the regex to split the string.

### featured_artists

In [14]:
invalid_elems = df[df['featured_artists'].apply(check_type_validity, expected_type=str)]
print(invalid_elems['featured_artists'])

nan_indexes = df.index[df['featured_artists'].isna()].tolist()
print(f"number of missing values in featured_artists: {len(nan_indexes)}")

5        NaN
6        NaN
8        NaN
9        NaN
10       NaN
        ... 
11159    NaN
11161    NaN
11163    NaN
11164    NaN
11165    NaN
Name: featured_artists, Length: 7649, dtype: object
number of missing values in featured_artists: 7649


this is expected, since not every track must have a featured artist. let's check if the featured artists of the column are the same listed in the full_title

In [18]:
df['featured_artists'] = (df['featured_artists']
                    .str.replace('\xa0', ' ', regex=False)
                    .str.replace('\u2019', "'", regex=False)
                    .str.replace('\u2018', "'", regex=False)
                    .str.replace('\u201c', '"', regex=False)
                    .str.replace('\u201d', '"', regex=False))

# \((?:Ft\.)\s*(.+?)\)\s*$ -> everything from "(Ft.)" until the closing ")"
extracted_featured = df['full_title'].str.extract(r'\((?:Ft\.)\s*(.+?)\)\s*$', expand=False).str.strip()
extracted_featured_normalized = extracted_featured.str.replace(' & ', ', ', regex=False)

def sort_splitted_by_comma(string):
    if pd.isna(string):
        return string
    ret = [a.strip() for a in string.split(',')]
    return ', '.join(sorted(ret))

extracted_sorted = extracted_featured_normalized.apply(sort_splitted_by_comma)
featured_sorted = df['featured_artists'].apply(sort_splitted_by_comma)

mismatches = df[
    ((extracted_sorted.notna()) & (featured_sorted.isna())) |
    ((extracted_sorted.isna()) & (featured_sorted.notna())) |
    ((extracted_sorted.notna()) & (featured_sorted.notna()) & (extracted_sorted != featured_sorted))
]

print(f"Rows with featured artists in full_title: {extracted_featured.notna().sum()}")
print(f"Rows with featured artists in column: {df['featured_artists'].notna().sum()}")
print(f"number of featured_artists mismatches (ignoring order): {len(mismatches)}\n")

if len(mismatches) > 0:
    print("Sample mismatches (actual differences, not just ordering):")
    mismatches_display = mismatches.copy()
    mismatches_display['featured_sorted'] = featured_sorted[mismatches.index]
    mismatches_display['extracted_sorted'] = extracted_sorted[mismatches.index]
    display(mismatches_display[['full_title', 'featured_sorted', 'extracted_sorted']].head(10))

Rows with featured artists in full_title: 3517
Rows with featured artists in column: 3517
number of featured_artists mismatches (ignoring order): 9

Sample mismatches (actual differences, not just ordering):


,full_title,featured_sorted,extracted_sorted
1566,Comunque vada... by Piotta (Ft. Turi & Compari),Turi & Compari,"Compari, Turi"
2427,Musica Vera by Mistaman (Ft. Stokka & MadBuddy),Stokka & MadBuddy,"MadBuddy, Stokka"
2461,Il giorno in cui ci siamo svegliati by Mistama...,Stokka & MadBuddy,"MadBuddy, Stokka"
2645,Lo Facciamo Così by Inoki (Ft. Stokka & MadBuddy),Stokka & MadBuddy,"MadBuddy, Stokka"
2844,Dreams by Johnny Marsiglia & Big Joe (Ft. Stok...,Stokka & MadBuddy,"MadBuddy, Stokka"
3301,Pace by Ghemon (Ft. Franco Negrè & Stokka & Ma...,"Franco Negrè, Stokka & MadBuddy","Franco Negrè, MadBuddy, Stokka"
3616,'O Sarracino by Neffa (Ft. Giuliano Palma & Th...,Giuliano Palma & The Bluebeaters,"Giuliano Palma, The Bluebeaters"
5576,CHIAGNE by Geolier (Ft. Lazza & Takagi & Ketra),"Lazza, Takagi & Ketra","Ketra, Lazza, Takagi"
9095,DONDURÈ by Dark Polo Gang (Ft. Ski & Wok),Ski & Wok,"Ski, Wok"


featured_artists is contained inside full_title with no errors. the missmatches are given from features made with group names including the character '&', which is what's used from the regex to split the string.